In [2]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [15]:
dataset_path = 'D:\\YapayZekaProje\\archive\\images\\train'
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [16]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for emotion in emotion_labels:
        emotion_folder = os.path.join(folder, emotion)
        for filename in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (48, 48))  # Resimleri 48x48 boyutuna getir
                images.append(img)
                labels.append(emotion)
    return np.array(images), np.array(labels)


In [17]:
X, y = load_images_from_folder(dataset_path)

In [18]:
X = X[..., np.newaxis]

In [19]:
label_encoder = LabelEncoder()
y = to_categorical(label_encoder.fit_transform(y))

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [35]:
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # 7 duygu sınıfı için
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [36]:
def train_model(X_train, y_train):
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
    return model

In [37]:
model = train_model(X_train, y_train)

Epoch 1/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.2302 - loss: 1.8405 - val_accuracy: 0.3116 - val_loss: 1.6800
Epoch 2/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.3317 - loss: 1.6871 - val_accuracy: 0.4074 - val_loss: 1.5400
Epoch 3/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.3889 - loss: 1.5699 - val_accuracy: 0.4471 - val_loss: 1.4410
Epoch 4/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.4224 - loss: 1.4847 - val_accuracy: 0.4744 - val_loss: 1.3907
Epoch 5/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.4415 - loss: 1.4448 - val_accuracy: 0.4794 - val_loss: 1.3684
Epoch 6/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.4679 - loss: 1.3905 - val_accuracy: 0.5022 - val_loss: 1.3139
Epoch 7/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.4844 - loss: 1.3534 - val_accuracy: 0.5091 - val_loss: 1.3055
Epoch 8/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.4906 - loss: 1

In [38]:
model.save('emotion_detection_cnn_model.h5')